In [5]:
import time
import torch
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score
from peft import PromptTuningConfig, get_peft_model, TaskType, PromptTuningInit

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
dataset = load_dataset("imdb")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "label"])

train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

In [11]:
virtual_tokens_list = [5, 20, 50, 100]

results_dict = {}

In [12]:

for num_virtual_tokens in virtual_tokens_list:
    print(f"\nЗапуск обучения с {num_virtual_tokens} виртуальными токенами")
    
    base_model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)
    base_model.config.pad_token_id = tokenizer.eos_token_id

    peft_config = PromptTuningConfig(
        task_type=TaskType.SEQ_CLS,
        num_virtual_tokens=num_virtual_tokens,
        prompt_tuning_init=PromptTuningInit.RANDOM,
        tokenizer_name_or_path="gpt2"
    )
    
    model = get_peft_model(base_model, peft_config)
    model.to(device)
    
    training_args = TrainingArguments(
        output_dir=f"./results/prompt_{num_virtual_tokens}",
        num_train_epochs=1,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        eval_strategy="epoch",
        logging_strategy="no",
        learning_rate=5e-5,
        weight_decay=0.01,
        report_to="none",
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )
    
    if torch.cuda.is_available():
        torch.cuda.reset_max_memory_allocated()
    
    start_time = time.time()
    trainer.train()
    elapsed_time = time.time() - start_time
    
    eval_results = trainer.evaluate()
    
    if torch.cuda.is_available():
        max_memory = torch.cuda.max_memory_allocated() / (1024 ** 2)
    else:
        max_memory = None

    results_dict[num_virtual_tokens] = {
        "training_time_sec": elapsed_time,
        "eval_results": eval_results,
        "max_gpu_memory_MB": max_memory,
    }
    
    print(f"Виртуальные токены: {num_virtual_tokens}")
    print(f"Время обучения: {elapsed_time:.2f} секунд")
    print("Метрики на валидации:", eval_results)
    if max_memory:
        print(f"Максимальное использование GPU памяти: {max_memory:.2f} MB")


Запуск обучения с 5 виртуальными токенами


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-12-691bac62544a>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:365: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(
GPT2ForSequenceClassification will not detect padding tokens in `inputs_embeds`. Results may be unexpected if using padding tokens in conjunction with `inputs_embeds.`


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.568673,0.509320,0.438492


Виртуальные токены: 5
Время обучения: 2963.51 секунд
Метрики на валидации: {'eval_loss': 1.5686726570129395, 'eval_accuracy': 0.50932, 'eval_f1': 0.4384921746462307, 'eval_runtime': 572.7175, 'eval_samples_per_second': 43.652, 'eval_steps_per_second': 43.652, 'epoch': 1.0}
Максимальное использование GPU памяти: 962.88 MB

Запуск обучения с 20 виртуальными токенами


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.091276,0.510280,0.396193


Виртуальные токены: 50
Время обучения: 3091.07 секунд
Метрики на валидации: {'eval_loss': 1.0912758111953735, 'eval_accuracy': 0.51028, 'eval_f1': 0.39619257889815895, 'eval_runtime': 598.5305, 'eval_samples_per_second': 41.769, 'eval_steps_per_second': 41.769, 'epoch': 1.0}
Максимальное использование GPU памяти: 1003.24 MB

Запуск обучения с 100 виртуальными токенами


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-12-691bac62544a>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:365: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.137965,0.510320,0.377204


Виртуальные токены: 100
Время обучения: 3655.14 секунд
Метрики на валидации: {'eval_loss': 1.137965202331543, 'eval_accuracy': 0.51032, 'eval_f1': 0.37720364195111994, 'eval_runtime': 707.1165, 'eval_samples_per_second': 35.355, 'eval_steps_per_second': 35.355, 'epoch': 1.0}
Максимальное использование GPU памяти: 1044.43 MB


In [13]:
print("\nСводка результатов по количеству виртуальных токенов:")
for vt, metrics in results_dict.items():
    print(f"Виртуальные токены: {vt} -> Время: {metrics['training_time_sec']:.2f} сек, "
          f"Метрики: {metrics['eval_results']}, "
          f"GPU память: {metrics['max_gpu_memory_MB']:.2f} MB" if metrics['max_gpu_memory_MB'] is not None else "")



Сводка результатов по количеству виртуальных токенов:
Виртуальные токены: 5 -> Время: 2963.51 сек, Метрики: {'eval_loss': 1.5686726570129395, 'eval_accuracy': 0.50932, 'eval_f1': 0.4384921746462307, 'eval_runtime': 572.7175, 'eval_samples_per_second': 43.652, 'eval_steps_per_second': 43.652, 'epoch': 1.0}, GPU память: 962.88 MB
Виртуальные токены: 20 -> Время: 3038.28 сек, Метрики: {'eval_loss': 1.011252522468567, 'eval_accuracy': 0.51152, 'eval_f1': 0.45588630183053674, 'eval_runtime': 593.2737, 'eval_samples_per_second': 42.139, 'eval_steps_per_second': 42.139, 'epoch': 1.0}, GPU память: 968.08 MB
Виртуальные токены: 50 -> Время: 3091.07 сек, Метрики: {'eval_loss': 1.0912758111953735, 'eval_accuracy': 0.51028, 'eval_f1': 0.39619257889815895, 'eval_runtime': 598.5305, 'eval_samples_per_second': 41.769, 'eval_steps_per_second': 41.769, 'epoch': 1.0}, GPU память: 1003.24 MB
Виртуальные токены: 100 -> Время: 3655.14 сек, Метрики: {'eval_loss': 1.137965202331543, 'eval_accuracy': 0.51032